# Algorithmic Trading - Assignment 3

In [1]:
# load in appropriate packages
import pandas as pd
import numpy as np

In [2]:
# load in and prepare data
data_train = pd.read_csv('Bond_MidModelTraining.csv')
data_test  = pd.read_csv('Bond_QuoteLive.csv')

data_train = data_train.ffill()
data_train = data_train.bfill() # because the first entry in some columns is a nan too

data_test = data_test.ffill()
#data_test = data_test.bfill() # same for the test set

# check if both sets are ordered properly by epochhours
#if data_train.iloc[:,0] == data_train.sort_values(0)

if not np.all(data_train.sort_values('epochhours').index == data_train.index):
    data_train = data_train.sort_values('epochhours', ignore_index=True)

if not np.all(data_test.sort_values('epochhours').index == data_test.index):
    data_test = data_test.sort_values('epochhours', ignore_index = True)

In [3]:
data_train.head()

,epochhours,last_price,firm_executable_bid,firm_executable_ask,market_estimate_bid,market_estimate_ask
0,1603087200,122.6405,122.335739,122.960435,122.314000,122.941544
1,1603087200,122.6405,122.335739,122.960435,122.314000,122.941544
2,1603090800,122.6405,122.374000,122.994429,122.314000,122.941544
3,1603090800,122.6405,122.374000,122.994429,122.288552,122.912063
4,1603094400,122.6405,122.374000,122.994429,122.386577,123.009203


In [ ]:
np.all(data_train.index == data_train.sort_values('epochhours').index)
np.all(data_test.index == data_test.sort_values('epochhours').index)

Now that the data is loaded in and the NaN's are taken care of, we can properly start the assignment